<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       BYOM In-Vantage Scoring with PMML and ONNX
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>Introduction</b></p>

<p style='font-size:16px;font-family:Arial'>In this notebook, we will show you how to work with the Bring Your Own Model (BYOM) pattern and BYOM In-Vantage Scoring. This pattern allows you to use whatever data science platform you want to perform model development and experimentation. You can use the vast majority of popular data science libraries and transformations. The only constraint is that you can convert it to one of the following open formats.</p>

- ONNX
- PMML
- H2O (MOJO)
- H2O (Driverless AI)

<p style='font-size:16px;font-family:Arial'>ONNX is become more popular by the day. It is a very efficient model format which was created and is maintained by Microsoft and its adoption by other companies and libraries as the standard open format is incresingly rapidly. While the name suggests it is primarily related to neural networks, it can be used with most sklearn libraries and algorithms.</p>


<p style='font-size:16px;font-family:Arial'>In this example, we will show you how you can develop in a notebook or other third-party tooling, produce a model and convert it to both `onnx` and `pmml` formats for deploying in Vantage with ModelOps.</p>

<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>Configure the Environment</b>

In [ ]:
%pip install xgboost scikit-learn nyoka skl2onnx onnxmltools

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>Libraries import</b></p>

In [1]:
import os
import pandas as pd
import getpass

from teradataml import (
    create_context, 
    remove_context,
    get_connection,
    DataFrame,
    PMMLPredict,
    configure)

In [25]:
# configure byom/val installation
configure.val_install_location = "VAL"
configure.byom_install_location = "MLDB"

<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>Connect to Vantage</b>

In [3]:
host = input("Host:")
username = input("Username:")
password = getpass.getpass("Password:")
database = input("Database (defaults to user):")

if not database:
    database = username

engine = create_context(host=host, 
                        username=username, 
                        password=password, 
                        logmech="TDNEGO",
                        database=database)


<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>Train Model</b>

In [23]:
from xgboost import XGBClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline

database = username

train_pdf = DataFrame.from_query(f"""
SELECT 
    F.*, D.hasdiabetes
FROM {database}.PIMA_PATIENT_FEATURES F 
JOIN {database}.PIMA_PATIENT_DIAGNOSES D
ON F.patientid = D.patientid
    WHERE D.patientid MOD 5 <> 0
""").to_pandas(all_rows=True)

features = ["NumTimesPrg", "Age", "PlGlcConc", "BloodP", "SkinThick", "TwoHourSerIns", "BMI", "DiPedFunc"]
target = "HasDiabetes"

# split data into X and y
X_train = train_pdf[features]
y_train = train_pdf[target]

model = Pipeline([('scaler', MinMaxScaler()),
                  ('xgb', XGBClassifier(eta=0.2, max_depth=6))])

model.fit(X_train, y_train)


Pipeline(steps=[('scaler', MinMaxScaler()),
                ('xgb',
                 XGBClassifier(base_score=None, booster=None, callbacks=None,
                               colsample_bylevel=None, colsample_bynode=None,
                               colsample_bytree=None, device=None,
                               early_stopping_rounds=None,
                               enable_categorical=False, eta=0.2,
                               eval_metric=None, feature_types=None, gamma=None,
                               grow_policy=None, importance_type=None,
                               interaction_constraints=None, learning_rate=None,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=6, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=None, n_jobs=None,
                               num_parallel_tree=None, ...))])

<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>Convert models</b>

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>Convert to PMML</b></p>

<p style = 'font-size:16px;font-family:Arial'>You can use the sklearn2pmml or the nyoka python libraries to convert to pmml. The nyoka is a python only package and so it is preferrable.</p> 

In [ ]:
from nyoka import xgboost_to_pmml

xgboost_to_pmml(pipeline=model, col_names=features, target_name=target, pmml_f_name="model.pmml")

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>Convert to ONNX</b></p>

<p style = 'font-size:16px;font-family:Arial'>We can also convert the model to onnx format. This is a bit more involved as the client libraries for converting from sklearn/xgboost to onnx are not yet as mature.</p>

In [ ]:
import numpy as np
from skl2onnx import to_onnx
from skl2onnx import convert_sklearn, to_onnx, update_registered_converter
from skl2onnx.common.shape_calculator import (
    calculate_linear_classifier_output_shapes,
    calculate_linear_regressor_output_shapes)
from onnxmltools.convert.xgboost.operator_converters.XGBoost import convert_xgboost
from onnxmltools.convert import convert_xgboost as convert_xgboost_booster

update_registered_converter(
    XGBClassifier, 'XGBoostXGBClassifier',
    calculate_linear_classifier_output_shapes, convert_xgboost,
    options={'nocl': [True, False], 'zipmap': [True, False, 'columns']})


model_onnx = to_onnx(model, X_train.astype(np.float32), target_opset=15)
with open("model.onnx", "wb") as f:
    f.write(model_onnx.SerializeToString())


<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>Import into ModelOps to Operationalize</b>

<p style = 'font-size:16px;font-family:Arial'>Go to the ModelOps UI and import this as a new model version. Then follow the workflow to deploy. Note that you can also import programatically via the ModelOps Python SDK.</p>

<p style = 'font-size:16px;font-family:Arial'>You may be wondering why you can't just directly insert the onnx or pmml model directly into the database table. And the answer is you can. However, with ModelOps, you get full governance around this model deployment, including data drift and model monitoring and alerting.</p>

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>View Published Models</b></p>

<p style = 'font-size:16px;font-family:Arial'>Once deployed via ModelOps, we can view the models published to vantage by querying the table they are published to. Note this information is available via the AOA APIs also.</p>



In [34]:
pd.options.display.max_colwidth = 250
pd.read_sql("SELECT TOP 2 * FROM Aoa_Byom_Models", get_connection())

,model_version,model_id,model_type,project_id,deployed_at,model
0,b343af15-b756-4046-bbc1-9b1988d2386f,234836a2-8e9d-420c-b5e8-33bc2d50c7b1,PMML,07876c6e-41f0-4995-8392-4d55017b235e,2024-01-26 13:48:43.300,"b'<?xml version=""1.0"" encoding=""UTF-8""?>\r\n<PMML xmlns=""http://www.dmg.org/PMML-4_4"" version=""4.4.1"">\r\n <Header copyright=""Copyright (c) 2021 Software AG"" description=""Default description"">\r\n <Application name=""Nyoka"" version=""5.5...."
1,3e15e241-728b-42fa-b929-dbc09fb32e69,c92cb63d-a90e-492b-9eb0-1d582e61bcc4,ONNX,07876c6e-41f0-4995-8392-4d55017b235e,2024-01-26 14:51:30.960,"b'\x08\x08\x12\x08skl2onnx\x1a\x061.16.0""\x07ai.onnx(\x002\x00:\xa9\xeb\x06\n\xb5\x01\n\x0bNumTimesPrg\n\x03Age\n\tPlGlcConc\n\x06BloodP\n\tSkinThick\n\rTwoHourSerIns\n\x03BMI\n\tDiPedFunc\x12\x0cconcatenated\x1a\x11FeatureVectorizer""\x11FeatureV..."


<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>On-Demand Scoring</b>

In [ ]:
model_version="b343af15-b756-4046-bbc1-9b1988d2386f"

model = DataFrame.from_query(f"""
    SELECT * FROM Aoa_Byom_Models 
    WHERE model_version = '{model_version}'
""")

preds = PMMLPredict(
        modeldata=model,
        newdata=DataFrame.from_query("SELECT * FROM Pima_Patient_Features WHERE patientid MOD 5 = 0"),
        accumulate=['PatientId'])

preds.result.to_pandas().head(10)

In [18]:
byom_db = "MLDB"

query = f"""
SELECT * FROM {byom_db}.PMMLPredict (
    ON (SELECT * FROM Pima_Patient_Features WHERE patientid MOD 5 = 0) AS DataTable
    ON (SELECT * FROM Aoa_Byom_Models 
            WHERE model_version='{model_version}') AS ModelTable DIMENSION
    USING
      Accumulate ('patientid')
) AS td;
"""

pd.read_sql(query, get_connection()).head(10)

,PatientId,prediction,json_report
0,545,1,"{""predicted_HasDiabetes"":1,""HasDiabetes_probability_0"":0.010192740711530202,""HasDiabetes_probability_1"":0.9898072592884698}"
1,600,0,"{""predicted_HasDiabetes"":0,""HasDiabetes_probability_0"":0.9582387894089346,""HasDiabetes_probability_1"":0.04176121059106547}"
2,265,1,"{""predicted_HasDiabetes"":1,""HasDiabetes_probability_0"":0.34224256582701496,""HasDiabetes_probability_1"":0.657757434172985}"
3,5,0,"{""predicted_HasDiabetes"":0,""HasDiabetes_probability_0"":0.8957612695619017,""HasDiabetes_probability_1"":0.1042387304380983}"
4,530,0,"{""predicted_HasDiabetes"":0,""HasDiabetes_probability_0"":0.9939569918849724,""HasDiabetes_probability_1"":0.006043008115027605}"
5,20,0,"{""predicted_HasDiabetes"":0,""HasDiabetes_probability_0"":0.7920320046307028,""HasDiabetes_probability_1"":0.20796799536929722}"
6,305,0,"{""predicted_HasDiabetes"":0,""HasDiabetes_probability_0"":0.9551838565109425,""HasDiabetes_probability_1"":0.04481614348905748}"
7,60,0,"{""predicted_HasDiabetes"":0,""HasDiabetes_probability_0"":0.9991887919016772,""HasDiabetes_probability_1"":8.112080983227844E-4}"
8,570,0,"{""predicted_HasDiabetes"":0,""HasDiabetes_probability_0"":0.9795150260088062,""HasDiabetes_probability_1"":0.02048497399119374}"
9,180,0,"{""predicted_HasDiabetes"":0,""HasDiabetes_probability_0"":0.9968760184705906,""HasDiabetes_probability_1"":0.0031239815294094113}"


<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>Custom BYOM Evaluation Logic</b>

<p style = 'font-size:16px;font-family:Arial'>You can define custom evaluation logic for BYOM models in ModelOps. This allows you to define your own charts, metrics etc that are to be created and captured as part of evaluation / comparison.</p>

In [28]:
from aoa.stats.stats import _capture_stats, _NpEncoder
import json
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)

train_df = DataFrame.from_query("""
SELECT 
    F.*, D.hasdiabetes 
FROM Pima_Patient_Features F
JOIN pima_patient_diagnoses D
    ON F.patientid = D.patientid 
    WHERE F.patientid MOD 5 <> 0
""")

data_stats = _capture_stats(df=train_df,
                            features=features,
                            targets=[target],
                            categorical=[target],
                            feature_metadata_fqtn=f"{database}.aoa_statistics_metadata")

with open("data_stats.json", 'w+') as f:
    json.dump(data_stats, f, indent=2, cls=_NpEncoder)

In [29]:
from sklearn import metrics
from teradataml import (
    get_context,
    DataFrame,
    PMMLPredict,
    configure
)
from aoa import (
    record_evaluation_stats,
    aoa_create_context,
    store_byom_tmp,
    ModelContext
)

import os
import json


def plot_confusion_matrix(cf, img_filename):
    import itertools
    import matplotlib.pyplot as plt
    plt.imshow(cf, cmap=plt.cm.Blues, interpolation='nearest')
    plt.colorbar()
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.xticks([0, 1], ['0', '1'])
    plt.yticks([0, 1], ['0', '1'])

    thresh = cf.max() / 2.
    for i, j in itertools.product(range(cf.shape[0]), range(cf.shape[1])):
        plt.text(j, i, format(cf[i, j], 'd'), horizontalalignment='center',
                 color='white' if cf[i, j] > thresh else 'black')

    fig = plt.gcf()
    fig.savefig(img_filename, dpi=500)
    plt.clf()


def evaluate(context: ModelContext, **kwargs):
    aoa_create_context()

    
    # this evaluation.py can hanlde both onnx and pmml. usually, you would only need to support one but for 
    # demo purposes, we will show with both as we produce both onnx and pmml in this notebook.
    
    import glob
    for file_name in glob.glob(f"{context.artifact_input_path}/model.*"):
        model_type = file_name.split(".")[-1]
    
    with open(f"{context.artifact_input_path}/model.{model_type}", "rb") as f:
        model_bytes = f.read()
        
    model = store_byom_tmp(get_context(), "byom_models_tmp", context.model_version, model_bytes)

    target_name = context.dataset_info.target_names[0]

    if model_type.upper() == "ONNX":
        byom_target_sql = "CAST(CAST(json_report AS JSON).JSONExtractValue('$.output_label[0]') AS INT)"
        mldb = os.environ.get("AOA_BYOM_INSTALL_DB", "MLDB")

        query = f"""
            SELECT sc.{context.dataset_info.entity_key}, {target_name}, sc.json_report
                FROM {mldb}.ONNXPredict(
                    ON ({context.dataset_info.sql}) AS DataTable
                    ON (SELECT model_version as model_id, model FROM byom_models_tmp) AS ModelTable DIMENSION
                    USING
                        Accumulate('{context.dataset_info.entity_key}', '{target_name}')
            ) sc;
        """

        predictions_df = DataFrame.from_query(query)
        
    elif model_type.upper() == "PMML":
        byom_target_sql = "CAST(CAST(json_report AS JSON).JSONExtractValue('$.predicted_HasDiabetes') AS INT)"
        
        pmml = PMMLPredict(
            modeldata=model,
            newdata=DataFrame.from_query(context.dataset_info.sql),
            accumulate=[context.dataset_info.entity_key, target_name])
        
        predictions_df = pmml.result

    predictions_df.to_sql(table_name="predictions_tmp", if_exists="replace", temporary=True)

    metrics_df = DataFrame.from_query(f"""
    SELECT 
        HasDiabetes as y_test, 
        {byom_target_sql} as y_pred
        FROM predictions_tmp
    """)
    metrics_df = metrics_df.to_pandas()

    y_pred = metrics_df[["y_pred"]]
    y_test = metrics_df[["y_test"]]

    evaluation = {
        'Accuracy': '{:.2f}'.format(metrics.accuracy_score(y_test, y_pred)),
        'Recall': '{:.2f}'.format(metrics.recall_score(y_test, y_pred)),
        'Precision': '{:.2f}'.format(metrics.precision_score(y_test, y_pred)),
        'f1-score': '{:.2f}'.format(metrics.f1_score(y_test, y_pred))
    }

    with open(f"{context.artifact_output_path}/metrics.json", "w+") as f:
        json.dump(evaluation, f)

    # create confusion matrix plot
    cf = metrics.confusion_matrix(y_test, y_pred)

    plot_confusion_matrix(cf, f"{context.artifact_output_path}/confusion_matrix")

    # calculate stats if training stats exist
    if os.path.exists(f"{context.artifact_input_path}/data_stats.json"):
        record_evaluation_stats(features_df=DataFrame.from_query(context.dataset_info.sql),
                                predicted_df=DataFrame("predictions_tmp"),
                                context=context)


In [31]:
from aoa import ModelContext, DatasetInfo

# Define the ModelContext to test with. The ModelContext is created and managed automatically by ModelOps 
# when it executes your code via CLI / UI. However, for testing in the notebook, you can define as follows

# define the evaluation dataset 
sql = """
SELECT 
    F.*, D.hasdiabetes 
FROM PIMA_PATIENT_FEATURES F 
JOIN PIMA_PATIENT_DIAGNOSES D
ON F.patientid = D.patientid
    WHERE D.patientid MOD 5 = 0
"""

feature_metadata =  {
    "database": database,
    "table": "aoa_statistics_metadata"
}

entity_key = "PatientId"
target_names = ["HasDiabetes"]
feature_names = ["NumTimesPrg", "PlGlcConc", "BloodP", "SkinThick", "TwoHourSerIns", "BMI", "DiPedFunc", "Age"]

dataset_info = DatasetInfo(sql=sql,
                           entity_key=entity_key,
                           feature_names=feature_names,
                           target_names=target_names,
                           feature_metadata=feature_metadata)

ctx = ModelContext(hyperparams={},
                   dataset_info=dataset_info,
                   artifact_output_path="/tmp",
                   artifact_input_path="./",
                   model_version="v1",
                   model_table="aoa_model_v1")


# drop volatile table from session if executing multiple times
try:
    get_context().execute(f"DROP TABLE byom_models_tmp")
except: 
    pass

evaluate(context=ctx)

# view evaluation results
with open(f"{ctx.artifact_output_path}/metrics.json") as f:
    print(json.load(f))

ValueError: artefact_output_path (/tmp) does not exist

In [32]:
DataFrame.from_query("SELECT PatientId, HasDiabetes, json_report FROM predictions_tmp")

ValueError: [Teradata][teradataml](TDML_2088) Encountered problem with the query. {}